### Recommender system: Day in the city

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyspark
from pyspark.mllib.recommendation import ALS
import requests
import time

In [ ]:
client_id = 'ua-7j9yr_CgcTD_4tQblcA'
api_key = '8dsXYq6nxsBx-XpawRnB6ONyWjpz-vpt0hkk7XGMSh2KfMuoH-KIN-yEow0MBFCuuP6gljF9qgFa58jXB9NuvpY52o6EiSgTX4yncXLPVbIGSaHQi1fBSAWySnJcXHYx'

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    df = pd.DataFrame(response.json()['businesses'])
    return df

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    dfs = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        dfs.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    df = pd.concat(dfs, ignore_index=True)
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

df = all_results(url_params, api_key)
print(len(df))
df.head()

In [14]:
from itertools import combinations, permutations
from bs4 import BeautifulSoup

1. Web Scrape common names

In [93]:
page = f'https://www.ssa.gov/oact/babynames/decades/names2010s.html'

# Make a get request to retrieve the page
html_page = requests.get(page)

# Pass the page content to beautiful soup for parsing
soup = BeautifulSoup(html_page.content, 'html.parser') 
pretty_soup = soup.prettify()

In [94]:
pretty_soup = pretty_soup.strip('').split('\n')

In [96]:
pretty_soup.index('         <th abbr="female number" style="text-align:right; background-color:white; color:black">')

226

In [97]:
pretty_soup = pretty_soup[226:]

In [125]:
a = ''.join(pretty_soup)
a = a.replace('</td>',"")
a = a.replace(' ',"")
a = a.split('<td>')

In [130]:
b = list(filter(lambda x: x.isalpha() , a))

In [134]:
b[:10]

['Noah',
 'Emma',
 'Jacob',
 'Sophia',
 'Mason',
 'Olivia',
 'Liam',
 'Isabella',
 'William',
 'Ava']

In [145]:
page = f'https://www.yelp.com/user_details?userid=3TWGb7-IY3xjfgr2Htsy8w'

# Make a get request to retrieve the page
html_page = requests.get(page)

# Pass the page content to beautiful soup for parsing
soup1 = BeautifulSoup(html_page.content, 'html.parser') 
pretty_soup1 = soup1.prettify()

In [147]:
pretty_soup1.split('\n')

['<!DOCTYPE HTML>',
 '<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->',
 '<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->',
 '<!--[if (gt IE 9)|!(IE)]><!-->',
 '<html class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml">',
 ' <!--<![endif]-->',
 ' <head>',
 '  <script>',
 '   (function() {',
 '                var main = null;',
 '',
 '                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name===

In [21]:
names = [''.join(p) for p in permutations('abc')]
names

['abc', 'acb', 'bac', 'bca', 'cab', 'cba']

### 2. EDA -  business.json

In [196]:
# reading in file
df = pd.read_json('business.json', lines = True) 
df.describe()

In [197]:
# Dropping Canadian cities

df.postal_code = df.postal_code.apply(lambda x: x if len(x) == 5 else np.nan)

In [198]:
# Dropping Null Attributes column
df = df[~df['postal_code'].isnull()]  
df = df[~df['attributes'].isnull()]    

In [209]:
# Limiting low frequency cities to 500. Below is the list of cities to keep

keep_city = list(pd.DataFrame(df.groupby('city').filter(lambda x : len(x)>500)['city'].value_counts()).reset_index()['index'])
print('Cities to keep:', keep_city)
df.drop(df[~df.city.isin(keep_city)].index, inplace=True)
df.head()


Cities to keep: ['Las Vegas', 'Phoenix', 'Charlotte', 'Scottsdale', 'Pittsburgh', 'Mesa', 'Henderson', 'Tempe', 'Chandler', 'Madison', 'Cleveland', 'Glendale', 'Gilbert', 'Peoria', 'North Las Vegas', 'Champaign', 'Surprise', 'Concord', 'Goodyear', 'Matthews', 'Avondale', 'Huntersville', 'Fort Mill']


,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{'GoodForKids': 'False'},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,None,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
4,"4209 Stuart Andrew Blvd, Ste F","{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...",HhyxOkGAM07SRYtlQ4wMFQ,"Plumbing, Shopping, Local Services, Home Servi...",Charlotte,"{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ...",1,35.190012,-80.887223,Queen City Plumbing,28217,4,4.0,NC
7,"4545 E Tropicana Rd Ste 8, Tropicana","{'RestaurantsPriceRange2': '3', 'GoodForKids':...",gbQN7vr_caG_A1ugSmGhWg,"Hair Salons, Hair Stylists, Barbers, Men's Hai...",Las Vegas,"{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",1,36.099872,-115.074574,Supercuts,89121,3,3.5,NV
8,5940 W Union Hills Dr,"{'RestaurantsPriceRange2': '2', 'ByAppointment...",Y6iyemLX_oylRpnr38vgMA,"Nail Salons, Beauty & Spas, Day Spas",Glendale,"{'Tuesday': '12:0-18:0', 'Wednesday': '10:0-18...",0,33.654815,-112.188568,Vita Bella Fine Day Spa,85308,8,5.0,AZ


In [ ]:
# Create DF with Name, ID & Address of restaurant
# Create DF with Name, ID, Attributes, Rating of restaurant